In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [ ]:
print('preparing data...')
df_x_train = pd.read_csv('template_train.csv',header=None,  skiprows = 1,index_col = 0)
df_y_train = pd.read_csv('y_train.csv', header=0, index_col = 0)
X = df_x_train.values
y = df_y_train['y'].values
df_x_test = pd.read_csv('template_test.csv', header=None,skiprows = 1, index_col = 0)
X_test = df_x_test.values

In [ ]:
feature_size=len(X[0])
max_wav = np.amax(np.vstack((X[:, 0:feature_size], X_test[:, 0:feature_size])))
min_wav = np.amin(np.vstack((X[:, 0:feature_size], X_test[:, 0:feature_size])))
X[:, 0:feature_size] = ((X[:,0:feature_size] - min_wav) / (max_wav - min_wav))
X_test[:, 0:feature_size] = ((X_test[:,0:feature_size] - min_wav) / (max_wav - min_wav))

In [ ]:
clf = svm.SVC( random_state=0,class_weight='balanced',kernel='rbf',gamma=1 / (1000 * X.std()))
clf.fit(X, y) 
cross_val_score(clf, X, y, scoring='f1_micro', cv=5)  

In [ ]:
y_pred =clf.predict(X_test)

In [ ]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,y_pred[i]))
f.close()